In [1]:
import torch

In [4]:
points = torch.tensor([[1,2],[3,4],[5,6]], dtype=torch.float32)
points

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])

In [5]:
# 简单说，和numpy一样
points[0,:]

tensor([1., 2.])

In [7]:
points[1:,:]

tensor([[3., 4.],
        [5., 6.]])

In [8]:
points[:,:1]

tensor([[1.],
        [3.],
        [5.]])

In [9]:
# 转换成numpy
points_1 = torch.ones(3,4)
points_np = points_1.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [11]:
points_1 = torch.from_numpy(points_np)
points_1

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [12]:
# 存储和读写

# 方式1
torch.save(points_1,'points_1.t')

# 方式2
with open('points_1.t', 'wb') as f:
    torch.save(points_1, f)

In [13]:
p = torch.load("points_1.t")
p

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [15]:
with open('points_1.t', 'rb') as f:
    p = torch.load(f)
p

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [16]:
! pip install h5py

In [17]:
import h5py

In [35]:
import gc

gc.collect()

# write
points_1 = torch.ones(3,4)

f = h5py.File('points_1.hdf5','w')
datasets = f.create_dataset('coords', data=points_1.numpy()) # coords是数据的名字
f.close()

OSError: Unable to create file (unable to truncate a file which is already open)

In [34]:
# read
f = h5py.File('points_1.hdf5','r')
dset = f['coords']
# dataset = dset.copy()
f.close()
dset

NameError: name 'dataset' is not defined

In [30]:
points = torch.from_numpy(dset[:])
print(points)
f.close()

ValueError: Not a dataset (not a dataset)

In [43]:
# 维度命名
# 注意names指的是每个维度的名字，而非是tensor的名字
points0 = torch.tensor([[1,2],[3,4]],names=['zero','one'])
points0

<ipython-input-43-3086ff7fb7d2>:3: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/core/TensorImpl.h:1791.)
  points0 = torch.tensor([[1,2],[3,4]],names=['zero','one'])


tensor([[1, 2],
        [3, 4]], names=('zero', 'one'))

In [2]:
img_t = torch.randn(3,4,4)
weights = torch.tensor([0.2,0.7,0.07])
img_t

tensor([[[ 1.5654e+00, -1.6547e+00, -7.8958e-05, -7.7911e-01],
         [-1.4816e+00,  1.4153e-01, -1.5157e+00, -7.1782e-01],
         [ 1.2068e+00,  1.6646e+00, -2.0244e-01,  1.7578e-02],
         [-1.5783e+00, -1.1721e+00, -1.2506e-01, -5.7383e-01]],

        [[ 1.2621e-01, -6.8400e-01, -7.4884e-02,  1.5777e-01],
         [ 1.2438e+00, -9.8327e-01,  5.3378e-01, -6.7107e-01],
         [-7.8754e-01, -5.7916e-01, -3.5757e-01,  2.2519e-01],
         [ 7.8575e-01,  1.6667e+00,  8.6964e-01,  8.2794e-01]],

        [[-1.1588e+00, -6.6581e-01,  2.1772e-01, -5.3457e-01],
         [ 2.8268e-02,  2.1361e-01,  4.1326e-01,  2.1068e+00],
         [-9.4296e-01,  1.3125e+00,  2.6666e+00, -4.1969e-01],
         [ 1.3522e+00, -1.7458e+00,  1.5880e-01, -1.2498e-02]]])

In [3]:
batch_t = torch.randn(2, 3,4,4)

In [4]:
img_gray_naive = img_t.mean(-3) # 仅在倒数第三个维度取平均，即去掉倒数第三个维度
print(img_gray_naive.shape)
img_gray_naive

torch.Size([4, 4])


tensor([[ 0.1776, -1.0015,  0.0476, -0.3853],
        [-0.0699, -0.2094, -0.1896,  0.2393],
        [-0.1746,  0.7993,  0.7022, -0.0590],
        [ 0.1865, -0.4171,  0.3011,  0.0805]])

In [6]:
# 加权平均
# 首先要升维和待处理张量一致的shape
# 还有unsqueeze_（带下划线），这种是pandas的inplace属性，pytorch所有函数都有这种下划线的对应函数代表直接替换
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
print(unsqueezed_weights.shape)
unsqueezed_weights

torch.Size([3, 1, 1])


tensor([[[0.2000]],

        [[0.7000]],

        [[0.0700]]])

In [7]:
img_weighted = img_t * unsqueezed_weights
img_weighted

tensor([[[ 3.1308e-01, -3.3094e-01, -1.5792e-05, -1.5582e-01],
         [-2.9633e-01,  2.8305e-02, -3.0315e-01, -1.4356e-01],
         [ 2.4136e-01,  3.3293e-01, -4.0487e-02,  3.5157e-03],
         [-3.1566e-01, -2.3443e-01, -2.5012e-02, -1.1477e-01]],

        [[ 8.8349e-02, -4.7880e-01, -5.2419e-02,  1.1044e-01],
         [ 8.7066e-01, -6.8829e-01,  3.7364e-01, -4.6975e-01],
         [-5.5128e-01, -4.0541e-01, -2.5030e-01,  1.5764e-01],
         [ 5.5003e-01,  1.1667e+00,  6.0875e-01,  5.7956e-01]],

        [[-8.1115e-02, -4.6606e-02,  1.5241e-02, -3.7420e-02],
         [ 1.9788e-03,  1.4953e-02,  2.8928e-02,  1.4747e-01],
         [-6.6007e-02,  9.1875e-02,  1.8666e-01, -2.9379e-02],
         [ 9.4651e-02, -1.2221e-01,  1.1116e-02, -8.7485e-04]]])

In [8]:
batch_t_weighted = batch_t * unsqueezed_weights
batch_t_weighted.shape

torch.Size([2, 3, 4, 4])

In [9]:
img_gray_weighted = img_weighted.sum(-3) # 同样会消去对应的维度
print(img_gray_weighted)
img_gray_weighted.shape

tensor([[ 0.3203, -0.8564, -0.0372, -0.0828],
        [ 0.5763, -0.6450,  0.0994, -0.4658],
        [-0.3759,  0.0194, -0.1041,  0.1318],
        [ 0.3290,  0.8100,  0.5949,  0.4639]])


torch.Size([4, 4])

In [11]:
# 一步到位（不推荐使用，有点抽象且不容易debug）
# hw, c 要对应，可以理解为在channel（c）加权求和，而height（h）和weight不变
img_gray_weighted_fancy = torch.einsum("...chw,c->...hw",img_t,weights)
print(img_gray_weighted_fancy.shape)
img_gray_weighted_fancy

torch.Size([4, 4])


tensor([[ 0.3203, -0.8564, -0.0372, -0.0828],
        [ 0.5763, -0.6450,  0.0994, -0.4658],
        [-0.3759,  0.0194, -0.1041,  0.1318],
        [ 0.3290,  0.8100,  0.5949,  0.4639]])

In [13]:
weights_named = torch.tensor([0.2,0.7,0.07],names=['channels'])
weights_named

tensor([0.2000, 0.7000, 0.0700], names=('channels',))

In [14]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 4, 4]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 4, 4]) (None, 'channels', 'rows', 'columns')


In [15]:
# 将weight和img张量按照名字对齐，这样就自动完成增维操作
weights_aligned = weights_named.align_as(img_named)
weights_aligned

tensor([[[0.2000]],

        [[0.7000]],

        [[0.0700]]], names=('channels', 'rows', 'columns'))

In [16]:
# 注意必须要对齐后才能相乘，不能直接乘

gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([4, 4]), ('rows', 'columns'))

In [ ]:
# 取消名字
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names